<a href="https://colab.research.google.com/github/slala2121/Triplet-net-keras/blob/COS597D/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import time
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import random
from itertools import permutations
from PIL import Image
import cv2
import scipy.io

import tensorflow as tf

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.preprocessing import image


In [0]:
data_source_dir='/content/drive/My Drive/COS597D_proj/data/animal'
source_path=os.path.join('drive','My Drive', 'Colab Notebooks')

In [0]:
# load human similarity judgment data

human_sim_data=scipy.io.loadmat(os.path.join(data_source_dir,'turkResults_CogSci2016.mat'))

data_order=human_sim_data['animals_big_idx']
human_sim_scores=human_sim_data['simMatrix']
human_sim_scores_normalized=human_sim_scores*1.0/np.amax(human_sim_scores)

# flattens the vector
human_sim_scores_tril=human_sim_scores[np.tril_indices(120,k=-1)]
human_sim_scores_normalized_tril=human_sim_scores_normalized[np.tril_indices(120,k=-1)]

In [0]:
# load corresponding images
images_ordered=sorted(os.listdir(os.path.join(data_source_dir,'images')))
num_images=len(images_ordered)

# this order corresponds to the order prescribed in data_order

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

# based on the lifted scheme paper
def create_deep_base_network(input_dim,loss_type,num_classes=0,transfer=False,freeze_weights=False):
  weights='imagenet' if transfer else None
  conv_base = ResNet50(weights=weights, include_top=False, input_shape=input_dim)

  if freeze_weights:
    for layer in conv_base.layers:
      layer.trainable=False
  
  model = models.Sequential()
  model.add(layers.Input(input_dim))
  model.add(conv_base)

  model.add(layers.Flatten())
  model.add(layers.BatchNormalization())

  if loss_type=='classification':
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))
  else:
    model.add(layers.Dense(64, activation=None))
    model.add(layers.Dropout(0.5))

  return model


def create_shallow_network(input_dim,loss_type,num_classes=0):
  model=tf.keras.Sequential()
  model.add(layers.Input(input_dim))
  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same'))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.BatchNormalization())

  if loss_type=='classification':
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers(Dense(num_classes,activation='softmax')))
  else:
    model.add(tf.keras.layers.Dense(256, activation=None))
    model.add(tf.keras.layers.Dropout(0.5))

  return model

def construct_model(model_type,input_dim,loss_type,num_classes,transfer=False,freeze_weights=False):
  if model_type=='shallow':
    model=create_shallow_network(input_dim,loss_type,num_classes)
  elif model_type=='deep':
    model=create_deep_base_network(input_dim,loss_type,num_classes,transfer,freeze_weights)

  if loss_type =='triplet':
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
  return model

def construct_loss(loss_type,margin):
  if loss_type=='triplet':
    loss=tfa.losses.TripletSemiHardLoss(margin=margin)
  elif loss_type=='lifted':
    loss=tfa.losses.LiftedStructLoss(margin=margin)
  elif loss_type=='classification':
    loss=tf.keras.losses.CategoricalCrossentropy()
  return loss

In [0]:
model_type='deep'
dataset_name='cifar10'
input_dim=(32,32,3)
train_mean=np.load(os.path.join(source_path,dataset_name,'train_mean.npy'))
loss_type='lifted'
num_classes=10
transfer=True
freeze_weights=False
margin=1.0

def preprocess_input(img):
  
  preprocessed_img=img.copy()
  preprocessed_img=preprocessed_img.astype('float32')
  preprocessed_img=preprocessed_img-train_mean
  preprocessed_img=preprocessed_img/255.

  return preprocessed_img

model=construct_model(model_type,input_dim,loss_type,num_classes)
filepath=os.path.join('final_%s_%s_transfer_%s_freeze_%s_margin_%s.h5'%(loss_type,model_type,transfer,freeze_weights,margin))
# filepath='final_classification_deep_transfer_True_freeze_False_margin_1.0.h5'
model_file_path=os.path.join(source_path,dataset_name,model_type,filepath)
model.load_weights(model_file_path)
embeddings_layer=model.layers[-2]
embeddings_layer_model=tf.keras.models.Model(model.input,embeddings_layer.output)
# model = tf.keras.models.load_model(model_file_path) # bug with metric learning

# baseline VGG-19
# model=tf.keras.applications.VGG19(weights='imagenet',include_top=True)
# embeddings_layer_model = models.Model(inputs=model.input, outputs=model.layers[-2].output)
# input_dim=(224,224,3)
# preprocess_input=tf.keras.applications.vgg19.preprocess_input






In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
Total params: 23,727,040
Trainable params: 23,669,824
Non-trainable params: 57,216
_________________________________________________________________


In [0]:
# image preprocess and compute the activations

# compute activations from penultimate layer from a classification network 

# compute image features in order of the images
images_features=[]
for image_fname in images_ordered:
  img = image.load_img(os.path.join(data_source_dir,'images',image_fname), target_size=input_dim)
  x = image.img_to_array(img)
  x = preprocess_input(x)
  x = np.expand_dims(x, axis=0) # reshape in batch format

  features = embeddings_layer_model.predict(x)[0]
  images_features.append(features)

images_features=np.array(images_features)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
# compute correlations bet. DNN feature similarities and human similarity scores

nn_sim_scores=cosine_similarity(images_features,images_features)
nn_sim_scores_tril=nn_sim_scores[np.tril_indices(120,k=-1)]

nn_sim_scores_normalized=nn_sim_scores*1.0/np.amax(nn_sim_scores)
nn_sim_scores_normalized_tril=nn_sim_scores_normalized[np.tril_indices(120,k=-1)]


np.corrcoef(human_sim_scores_normalized_tril,nn_sim_scores_normalized_tril)**2

array([[1.        , 0.00152039],
       [0.00152039, 1.        ]])

In [0]:
# run MDS

from sklearn import manifold
import time

mds = manifold.MDS(n_components=2, metric=True, max_iter=10000, eps=1e-100,
                    dissimilarity="precomputed")
mds_human_sim_embed_coords=nmds.fit_transform(human_sim_scores_normalized)
nmds = manifold.MDS(n_components=2, metric=False, max_iter=10000, eps=1e-100)
nmds_human_sim_embed_coords=nmds.fit_transform(mds_human_sim_embed_coords)

# visualize NMDS

from matplotlib.offsetbox import OffsetImage, AnnotationBbox


fig,ax=plt.subplots(figsize=(5,5))

for image_index,image_fname in enumerate(images_ordered):
  img = image.load_img(os.path.join(data_source_dir,'images',image_fname), target_size=(224, 224))
  img = image.img_to_array(img)
  img*=1.0/np.amax(img)

  im = OffsetImage(img, zoom=0.1)
  im_nmds_coords=(nmds_human_sim_embed_coords[image_index][0],
                  nmds_human_sim_embed_coords[image_index][1])
  ab = AnnotationBbox(im, im_nmds_coords,frameon=False)
  ax.add_artist(ab)

ax.autoscale()
plt.tight_layout()

(120, 2)
